In [86]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [87]:
def _parse_line(line):

    rx_dict = {
    'token': re.compile(r'var token = "(?P<token>.*)"\n'),
    'url': re.compile(r'var url = "(?P<url>.*)"\n'),
    'org': re.compile(r'var org = "(?P<org>.*)"\n'),
    'bucket': re.compile(r'var bucket = "(?P<bucket>.*)"\n'),
    }   

    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

filepath = '/root/flexi-pipe/config.go'
# open the file and read through it line by line
with open(filepath, 'r') as file_object:
    line = file_object.readline()
    while line:
        # at each line check for a match with a regex
        key, match = _parse_line(line)

        if key == 'token':
            token = match.group('token')
        elif key == 'url':
            url = match.group('url')
        elif key == 'org':
            org = match.group('org')
        elif key == 'bucket':
            bucket = match.group('bucket')
        
        line = file_object.readline()
# url="http://192.168.20.58:8086"
url = "http://localhost:8086"

In [159]:
start = 1696343921
# # end = 1705735851
# start = 1707740868
end = 1708065415

node = "stoleman"

In [160]:
def experiment(start_time, end_time, filepath):
    # Retrieve experiments data from csv
    data = pd.read_csv(filepath, header=None)
    df = pd.DataFrame(data)

    #Rename columns
    experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "parameter", 5: "d", 6: "dlo", 7: "dhi", 8: "dscore", 9: "dlazy", 10: "dout", 11: "gossipFactor", 12: "initialDelay", 13: "interval"}, errors='raise')

    #Correct timestamp
    experiments["start"] = experiments["start"].str.slice(0, 27)
    experiments["end"] = experiments["end"].str.slice(0, 27)

    #String to timestamp
    # experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    # experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
    experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


    experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
    experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

    #Drop fields we don't mneed for the moment
    exp = experiments.drop(columns=["runtime", "initialDelay"]).sort_values(by=["start"])

    #Get times for different intervals
    # intervals = exp["interval"].drop_duplicates().sort_values().reset_index(drop=True)
    # intervals.head(10)

    expTime = exp[exp['startUnix'].astype(int).between(start_time, end_time)]
    # expTime['experiment'] = expTime.index
    expTime = expTime.reset_index().rename({'index':'experiment'}, axis = 'columns')

    return expTime

experiments = experiment(start, end, '../../Datasets/experiments_raw.csv')
experiments.to_csv('../../Datasets/experiments_filtered.csv')
experiments.tail(200)

/tmp/ipykernel_3385/3383679728.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
/tmp/ipykernel_3385/3383679728.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
329,712,2024-01-18 23:49:33.9823415,2024-01-19 00:19:33.9854755,unl,random28,20,18,23,8,4,16,0.50,1.00,1705621773,1705623573
330,713,2024-01-19 00:23:08.5330606,2024-01-19 00:53:08.5417381,unl,random28,20,18,23,8,4,16,0.50,1.00,1705623788,1705625588
331,714,2024-01-19 00:56:42.9001595,2024-01-19 01:26:42.9031478,unl,random29,18,20,8,4,6,4,0.25,3.00,1705625802,1705627602
332,715,2024-01-19 01:30:16.3550833,2024-01-19 02:00:16.3582969,unl,random29,18,20,8,4,6,4,0.25,3.00,1705627816,1705629616
333,716,2024-01-19 02:03:50.1184798,2024-01-19 02:33:50.1214603,unl,random29,18,20,8,4,6,4,0.25,3.00,1705629830,1705631630
334,717,2024-01-19 02:37:23.5929531,2024-01-19 03:07:23.5977598,unl,random30,15,4,20,8,2,8,1.00,1.00,1705631843,1705633643
335,718,2024-01-19 03:10:57.9766699,2024-01-19 03:40:57.9794300,unl,random30,15,4,20,8,2,8,1.00,1.00,1705633857,1705635657
336,719,2024-01-19 03:44:32.5404468,2024-01-19 04:14:32.5472689,unl,random30,15,4,20,8,2,8,1.00,1.00,1705635872,1705637672
337,720,2024-01-19 04:18:07.9259423,2024-01-19 04:48:07.9860640,unl,random31,8,6,18,4,4,4,0.25,0.50,1705637887,1705639687
338,721,2024-01-19 04:51:40.8244720,2024-01-19 05:21:40.8270549,unl,random31,8,6,18,4,4,4,0.25,0.50,1705639900,1705641700


In [161]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [162]:
def from_influx_count(url, token, org, start_time, end_time,grouping_key):
    # start_time = 1693222601 
    # end_time = 1693228430    
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)
        
        # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()
        
    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                       ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                        '|> filter(fn: (r) => r._measurement == "deliverMessage") '
                        '|> group(columns: ["_measurement", "_field"], mode: "by") '
                        '|> count()')
    client.close()

    if data_frame.empty:
        count = 0
    else:
        count = data_frame["_value"].min().astype(int)
    return count

In [163]:
#Validate data
validate = pd.DataFrame()

for index, row in experiments.iterrows():
    count = from_influx_count(url, token, org, row["startUnix"], row["endUnix"],"_measurement")
    d = {'experiment': [row['experiment']], 'count': [count], 'topology': [row["topology"]], 'd': [row["d"]],'dhi': [row["dhi"]],'dlo': [row["dlo"]],'dlazy': [row["dlazy"]],
        'dscore': [row["dscore"]],'dout': [row["dout"]],'gossipFactor': [row["gossipFactor"]],'interval': [row["interval"]],}
    aux = pd.DataFrame(data=d)
    validate = pd.concat([validate, aux])

# validate.head(200)

exps = experiments.merge(validate, on=['experiment', 'topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
exps = exps.loc[exps["count"]>0]

exps.head(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix,count
2,385,2023-10-04 13:25:01.0752251,2023-10-04 13:40:01.0774462,unl,reference,8,6,12,4,8,2,0.25,1.0,1696425901,1696426801,46733
3,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498,unl,reference,8,6,12,4,8,2,0.25,1.0,1696429128,1696430028,128631
6,389,2023-10-04 17:00:47.8475357,2023-10-04 17:15:47.8502305,unl,reference,8,6,12,4,8,2,0.25,1.0,1696438847,1696439747,18248
7,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,reference,8,6,12,4,8,2,0.25,1.0,1696445523,1696447323,344487
8,391,2023-10-04 19:25:37.1533399,2023-10-04 19:55:37.1555856,unl,reference,8,6,12,4,8,2,0.25,1.0,1696447537,1696449337,244016
9,392,2023-10-04 19:59:12.6589825,2023-10-04 20:29:12.6608307,unl,reference,8,6,12,4,8,2,0.25,1.0,1696449552,1696451352,130775
10,393,2023-10-04 20:32:47.3098616,2023-10-04 21:02:47.3149178,unl,informed,8,6,12,4,8,2,0.25,3.0,1696451567,1696453367,194668
11,394,2023-10-04 21:06:22.7755634,2023-10-04 21:36:22.7788800,unl,informed,8,6,12,4,8,2,0.25,3.0,1696453582,1696455382,165538
12,395,2023-10-04 21:39:58.6528165,2023-10-04 22:09:58.6545536,unl,informed,8,6,12,4,8,2,0.25,3.0,1696455598,1696457398,64772
13,396,2023-10-04 22:13:33.8146704,2023-10-04 22:43:33.8174094,unl,informed0,16,8,20,16,8,4,0.50,1.0,1696457613,1696459413,141092


In [164]:
#Remove fault executions

#get median
median = exps.drop(columns=['start','end','startUnix','endUnix', 'parameter', 'experiment']).drop_duplicates()
median = median.groupby(['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval']).agg({'count':['median', 'std', 'mean']})
median.columns = median.columns.droplevel(0)
median.reset_index(inplace=True)
median = median.loc[median['median'] > 0]

median.head(50)

,topology,d,dhi,dlo,dlazy,dscore,dout,gossipFactor,interval,median,std,mean
0,general,2,4,1,3,2,5,0.15,3.00,42318.0,59845.275319,42318.000000
1,general,3,5,2,2,2,2,0.50,1.00,63388.0,14596.972643,58323.666667
2,general,3,7,2,20,9,2,0.25,1.00,22025.5,28869.048556,22025.500000
3,general,4,7,2,9,10,2,0.50,1.00,62968.0,39320.030192,71566.333333
4,general,5,7,4,2,3,2,0.25,1.00,84236.0,33491.111970,95775.000000
5,general,5,20,3,9,7,4,0.50,1.00,179434.0,72252.912267,145625.000000
6,general,6,8,3,2,2,4,0.25,0.50,37796.5,42383.273358,37796.500000
7,general,6,8,4,2,2,2,0.25,1.00,62045.5,31217.900784,63183.833333
8,general,6,8,4,2,2,4,0.25,0.50,38482.0,19105.694839,27177.600000
9,general,6,8,4,2,2,4,0.25,3.00,11592.0,61427.235493,41103.333333


In [165]:
#Validate the data

validation = exps.merge(median, on=['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
validated = validation[(validation['count'] >= validation['mean']-(0.15*validation['std']))]
validated = validated.drop(columns=['startUnix', 'endUnix'])
validated.dropna()

validated.head(100)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,count,median,std,mean
3,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,reference,8,6,12,4,8,2,0.25,1.00,344487,142423.0,149514.670084,208016.555556
4,391,2023-10-04 19:25:37.1533399,2023-10-04 19:55:37.1555856,unl,reference,8,6,12,4,8,2,0.25,1.00,244016,142423.0,149514.670084,208016.555556
7,505,2023-11-07 18:12:32.6142390,2023-11-07 18:42:32.6167070,unl,reference,8,6,12,4,8,2,0.25,1.00,408150,142423.0,149514.670084,208016.555556
8,506,2023-11-07 18:46:01.6133827,2023-11-07 19:16:01.6158393,unl,reference,8,6,12,4,8,2,0.25,1.00,408686,142423.0,149514.670084,208016.555556
9,393,2023-10-04 20:32:47.3098616,2023-10-04 21:02:47.3149178,unl,informed,8,6,12,4,8,2,0.25,3.00,194668,162285.0,90760.806400,175292.500000
10,394,2023-10-04 21:06:22.7755634,2023-10-04 21:36:22.7788800,unl,informed,8,6,12,4,8,2,0.25,3.00,165538,162285.0,90760.806400,175292.500000
12,507,2023-11-07 19:19:31.9016456,2023-11-07 19:49:31.9049682,unl,informed,8,6,12,4,8,2,0.25,3.00,337252,162285.0,90760.806400,175292.500000
15,396,2023-10-04 22:13:33.8146704,2023-10-04 22:43:33.8174094,unl,informed0,16,8,20,16,8,4,0.50,1.00,141092,131035.5,74389.509136,106159.333333
16,397,2023-10-04 22:47:09.3837110,2023-10-04 23:17:09.3868442,unl,informed0,16,8,20,16,8,4,0.50,1.00,125279,131035.5,74389.509136,106159.333333
18,510,2023-11-07 21:00:03.5598397,2023-11-07 21:30:03.5622902,unl,informed0,16,8,20,16,8,4,0.50,1.00,136792,131035.5,74389.509136,106159.333333


In [166]:
#Limit extraction to 15 minutes

experiments = validated.drop(columns=['count','median', 'mean','std'])#.rename(columns={'start':'originalStart','end':'originalEnd'})

# experiments['originalStart'] = pd.to_datetime(experiments["originalStart"], format='mixed')
# experiments['originalEnd'] = pd.to_datetime(experiments["originalEnd"], format='mixed')

# experiments['start'] = experiments['originalStart'] + pd.Timedelta(hours=0, minutes=7, seconds=30)
# experiments['end'] = experiments['originalEnd'] - pd.Timedelta(hours=0, minutes=7, seconds=30)

experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed").astype('int64') / 10**9
experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed").astype('int64') / 10**9

experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

experiments.to_csv('../../Datasets/experiments_valid.csv')
experiments.head(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
3,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,reference,8,6,12,4,8,2,0.25,1.0,1696445523,1696447323
4,391,2023-10-04 19:25:37.1533399,2023-10-04 19:55:37.1555856,unl,reference,8,6,12,4,8,2,0.25,1.0,1696447537,1696449337
7,505,2023-11-07 18:12:32.6142390,2023-11-07 18:42:32.6167070,unl,reference,8,6,12,4,8,2,0.25,1.0,1699380752,1699382552
8,506,2023-11-07 18:46:01.6133827,2023-11-07 19:16:01.6158393,unl,reference,8,6,12,4,8,2,0.25,1.0,1699382761,1699384561
9,393,2023-10-04 20:32:47.3098616,2023-10-04 21:02:47.3149178,unl,informed,8,6,12,4,8,2,0.25,3.0,1696451567,1696453367
10,394,2023-10-04 21:06:22.7755634,2023-10-04 21:36:22.7788800,unl,informed,8,6,12,4,8,2,0.25,3.0,1696453582,1696455382
12,507,2023-11-07 19:19:31.9016456,2023-11-07 19:49:31.9049682,unl,informed,8,6,12,4,8,2,0.25,3.0,1699384771,1699386571
15,396,2023-10-04 22:13:33.8146704,2023-10-04 22:43:33.8174094,unl,informed0,16,8,20,16,8,4,0.50,1.0,1696457613,1696459413
16,397,2023-10-04 22:47:09.3837110,2023-10-04 23:17:09.3868442,unl,informed0,16,8,20,16,8,4,0.50,1.0,1696459629,1696461429
18,510,2023-11-07 21:00:03.5598397,2023-11-07 21:30:03.5622902,unl,informed0,16,8,20,16,8,4,0.50,1.0,1699390803,1699392603


In [167]:
data = pd.read_csv('../../Datasets/experiments_valid.csv',header=0, index_col=0)
experiments = pd.DataFrame(data)

experiments.tail(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
409,891,2024-02-15 19:25:21.9807959,2024-02-15 19:55:21.9832958,validator,random33,19,8,23,4,8,2,0.25,3.00,1708025121,1708026921
412,898,2024-02-15 23:20:24.1114255,2024-02-15 23:50:24.1747705,validator,random5,8,6,16,4,8,2,0.25,0.25,1708039224,1708041024
413,899,2024-02-15 23:53:58.9555104,2024-02-16 00:23:59.0175630,validator,random5,8,6,16,4,8,2,0.25,0.25,1708041238,1708043039
415,901,2024-02-16 01:01:07.9360686,2024-02-16 01:31:07.9411590,validator,random6,16,8,20,6,8,2,0.25,1.00,1708045267,1708047067
416,902,2024-02-16 01:34:42.7924620,2024-02-16 02:04:42.7974999,validator,random6,16,8,20,6,8,2,0.25,1.00,1708047282,1708049082
418,904,2024-02-16 02:41:52.3057095,2024-02-16 03:11:52.3106811,validator,random7,10,8,16,4,8,4,0.25,1.00,1708051312,1708053112
419,905,2024-02-16 03:15:26.8680015,2024-02-16 03:45:26.8705969,validator,random7,10,8,16,4,8,4,0.25,1.00,1708053326,1708055126
421,907,2024-02-16 04:22:35.4565262,2024-02-16 04:52:35.4619071,validator,random8,10,8,16,4,8,2,0.15,1.00,1708057355,1708059155
422,908,2024-02-16 04:56:10.5331406,2024-02-16 05:26:10.5389618,validator,random8,10,8,16,4,8,2,0.15,1.00,1708059370,1708061170
423,909,2024-02-16 05:29:45.3956405,2024-02-16 05:59:45.3984223,validator,reference,8,6,12,4,8,2,0.25,1.00,1708061385,1708063185


In [168]:
def from_influx_message(url, token, org, start_time, end_time):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "message" and r.node == "'+node+'" and r._field == "type" and (r._value == 11 or r._value == 12 or r._value == 3 or r._value == 2)) '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')

    client.close()
    
    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    if data_frame.empty == False:
        data_frame.reset_index(inplace=True)
        df = data_frame[['_time', 'type']].sort_values(by=["_time"]).reset_index(drop=True)
        df["_time"] = pd.to_datetime(df["_time"])
    else:
        df = pd.DataFrame()

    return df

In [169]:
# message = pd.DataFrame()
execs = experiments#.drop(columns=['originalStart', 'originalEnd'])
# final=final.loc[final["identifier"] == 1]

for idx, exp in execs.iterrows():
    # print(exp["startUnix"], exp["endUnix"],exp['experiment'])
    query_mess = from_influx_message(url, token, org, exp["startUnix"], exp["endUnix"])
    if query_mess.empty == False:
        query_mess["experiment"] = exp["experiment"]
    
        query_mess["start"] = exp["start"]
        query_mess["end"] = exp["end"]
                
        query_mess["_time"] = pd.to_datetime(query_mess["_time"])
            # message = pd.concat([message,query_mess])
    
        query_mess.to_csv('../../Datasets/partialTables/mess_'+node+'_'+str(exp["experiment"])+'.csv')
        
    # del query_mess
    gc.collect()


In [170]:
#Get list of peer ids
def from_influx_peerID(url, token, org, start_time, end_time):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "message" and r.node == "'+node+'" and r._field == "type" and (r._value == 11 or r._value == 12 or r._value == 3 or r._value == 2)) '
                                        ' |> group(columns: ["_tag"], mode: "by") '
                                        ' |> distinct(column: "peerID")')
                                        # ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')

    client.close()
    
    if data_frame.empty == False:
        data_frame.reset_index(inplace=True)
        df = data_frame[['_value']].rename(columns={"_value":"peerID"})
        # df["_time"] = pd.to_datetime(df["_time"])
    else:
        df = pd.DataFrame()

    return df

In [171]:
execs = experiments#.drop(columns=['originalStart', 'originalEnd'])
# final=final.loc[final["identifier"] == 1]
peer = pd.DataFrame()

for idx, exp in execs.iterrows():
    # print(exp["startUnix"], exp["endUnix"], exp["experiment"])
    query_mess = from_influx_peerID(url, token, org, exp["startUnix"], exp["endUnix"])
    if query_mess.empty == False:
        peer = pd.concat([peer,query_mess])

if peer.empty == False:
    peer=peer.reset_index()
    peer=peer.drop_duplicates()
    peerID = peer["peerID"].item()
    print(peerID)
    
peer.head(100)

ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0bcxmA=


,index,peerID
0,0,ACQIARIguLx6PkCG/GbhqGyJR3noZ4kxHLDCpZhJnTh8I0...


In [172]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'" and r.peerID == "'+str(peerID)+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    if data_frame.empty == False:
        data_frame.reset_index(inplace=True)
        df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
        df["_time"] = pd.to_datetime(df["_time"])
    else:
        df = data_frame

    return df

In [173]:
# gossip = pd.DataFrame()
execs = experiments#.drop(columns=['originalStart', 'originalEnd'])

for idx, exp in execs.iterrows():
    # print(exec["startUnix"], exec["endUnix"])
    query_iwant = from_influx(url, token, org, "iwant", exp["startUnix"], exp["endUnix"],"_measurement")

    if query_iwant.empty == False:
        query_iwant["experiment"] = exp["experiment"]
        query_iwant["_time"] = pd.to_datetime(query_iwant["_time"])

        query_iwant.to_csv('../../Datasets/partialTables/iwant_'+node+'_'+str(exp["experiment"])+'.csv')

        del query_iwant

    query_ihave = from_influx(url, token, org, "ihave", exp["startUnix"], exp["endUnix"],"_measurement")
    if query_ihave.empty == False:
        query_ihave["experiment"] = exp["experiment"]
        query_ihave["_time"] = pd.to_datetime(query_ihave["_time"])

        query_ihave.to_csv('../../Datasets/partialTables/ihave_'+node+'_'+str(exp["experiment"])+'.csv')
        
        del query_ihave

    gc.collect()

In [174]:
execs = experiments#.drop(columns=['originalStart', 'originalEnd'])

for idx, exp in execs.iterrows():
    query_rpc = from_influx(url, token, org, "recvRPC", exp["startUnix"], exp["endUnix"],"_measurement")
    
    query_rpc["experiment"] = exp["experiment"]
    
    query_rpc["start"] = exp["start"]
    query_rpc["end"] = exp["end"]
            
    query_rpc["_time"] = pd.to_datetime(query_rpc["_time"])
    
    query_rpc.to_csv('../../Datasets/partialTables/rpc_'+node+'_'+str(exp["experiment"])+'.csv')
    
    del query_rpc
    gc.collect()
        

In [175]:
for idx, exec in execs.iterrows():
    # print(exec["startUnix"], exec["endUnix"])
    query_mess = from_influx(url, token, org, "deliverMessage", exec["startUnix"], exec["endUnix"],"messageID")
    query_pub = from_influx(url, token, org, "publishMessage", exec["startUnix"], exec["endUnix"],"messageID")

    if query_mess.empty == False:
        query_mess["experiment"] = exec["experiment"]
        query_mess["_time"] = pd.to_datetime(query_mess["_time"])
        query_mess.to_csv('../../Datasets/partialTables/mess2_'+node+'_'+str(exec["experiment"])+'.csv')

    if query_pub.empty == False:
        query_pub["experiment"] = exec["experiment"]
        query_pub["_time"] = pd.to_datetime(query_pub["_time"])
        query_pub.to_csv('../../Datasets/partialTables/pub_'+node+'_'+str(exec["experiment"])+'.csv')

    del query_pub
    del query_mess

    gc.collect()